In [1]:
import fractions
c = "3  1/2"
def mixed_to_float(x):
    return float(sum(fractions.Fraction(term) for term in x.split()))
mixed_to_float(c)

3.5

In [2]:
import json
path = 'recipeDB.json'
with open(path) as f:
    # load the data
    json_data = json.load(f)

indian_dict = json_data["anyToIndian"]
style_dict = json_data["styles"]
health_dict = json_data["healthyToUnhealthy"]

In [3]:
from bs4 import BeautifulSoup

import requests


class RecipeFetcher:

    search_base_url = 'https://www.allrecipes.com/search/results/?wt=%s&sort=re'

    def search_recipes(self, keywords): 
        search_url = self.search_base_url %(keywords.replace(' ','+'))

        page_html = requests.get(search_url)
        page_graph = BeautifulSoup(page_html.content,features = "lxml")

        return [recipe.a['href'] for recipe in\
               page_graph.find_all('div', {'class':'grid-card-image-container'})]

    def scrape_recipe(self, recipe_url):
        results = {}

        page_html = requests.get(recipe_url)
        page_graph = BeautifulSoup(page_html.content,features = "lxml")
        results['ingredients'] = [ingredient.text for ingredient in\
                                  page_graph.find_all('span', {'itemprop':'recipeIngredient'})]
        
        results['labels'] = [ingredient.text for ingredient in\
#                                   page_graph.find_all('span', {'itemprop':'recipeIngredient'})
                                  page_graph.find_all('span', {'data-id':'0'})]
        
        results['directions'] = [direction.text.strip() for direction in\
                                 page_graph.find_all('span', {'class':'recipe-directions__list--item'})
                                 if direction.text.strip()]
        for i in results['ingredients']:
            if i in results['labels']:
                results['ingredients'].remove(i)
                
        return results

    
rf = RecipeFetcher()
paneer_makhani = rf.search_recipes('italian meatloaf')[0]
recipe = (rf.scrape_recipe(paneer_makhani))
recipe

{'ingredients': ['1/2 cup sun-dried tomatoes',
  '1 cup hot water',
  'cooking spray',
  '1 pound ground beef',
  '1 cup seasoned bread crumbs',
  '3/4 cup finely chopped onion',
  '3/4 cup chopped fresh basil',
  '1/2 cup ketchup',
  '2 ounces shredded sharp provolone cheese',
  '2 egg whites',
  '2 cloves garlic, chopped'],
 'labels': [],
 'directions': ['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.',
  'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
  'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together in a bowl; pack into prepared loaf pan.',
  'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']}

In [4]:
import re
dic = {}

for i in (recipe['ingredients']):
    dic[i] ={}
    x=re.findall('\w+ed ',i)
    if x!=[]:
        dic[i] = {'Preparation':x}
        
for i in (recipe['ingredients']):
    if 'egg' in i:
        pass
    #print(i)
    else:
        if '(' in i and '%' not in i:
            x=re.findall('\(\d+ (\w+)\)',i)
            if x!=[]:
                dic[i]['Measurement'] = x
        elif '%' in i:
            x=re.findall('\d+ (\w+)',i)
            if x!=[]:
                dic[i]['Measurement'] = x
        else:
            x = re.findall('\d+ (\w+) \w+',i)
            if x!=[]:
                dic[i]['Measurement']=x

for i in (recipe['ingredients']):
    if '(' in i and '%' not in i:
        x=re.findall('\((\d+) \w+\)',i)
        if x!=[]:
            dic[i]['Quantity'] = mixed_to_float(x[0])
    elif '%' in i:
        x=re.findall('(\d+) \w+',i)
        if x!=[]:
            dic[i]['Quantity'] = mixed_to_float(x[0])
    else:    
        if '/' in i:
            x=re.findall('(\d*\s*\d\/\d)+',i)
            if x!=[]:
                dic[i]['Quantity'] = mixed_to_float(x[0])

        else:
            x = re.findall('(\d+) \w',i)
            if x!=[]:
                dic[i]['Quantity']=mixed_to_float(x[0])
                
for i in (recipe['ingredients']):
    if 'egg' in i:
        x = re.findall('\d+ (.*)',i)
        dic[i]['Ingredient Name'] = x[0]
    else:
        x = i
        if ')' in i:

            x=re.sub('.*\(.*\) \w+','',x)
    #         if x!=[]:
    #             dic[i]['Quantity'] = x    
        if '/' in i:
                x=re.sub('\/','',x)
    #             if x!=[]:
    #                 dic[i]['Quantity'] = mixed_to_float(x[0])
        x =re.sub('\d+','',x)
        try:
            x = re.sub(dic[i]['Measurement'][0],'',x)
            x = re.sub(dic[i]['Preparation'][0],'',x)
        except:
            x = x
        x = re.sub('\,.*|to taste','',x).lstrip(' ')
        dic[i]['Ingredient Name'] = x
dic

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '],
  'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'sun-tomatoes'},
 '1 cup hot water': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'hot water'},
 'cooking spray': {'Ingredient Name': 'cooking spray'},
 '1 pound ground beef': {'Measurement': ['pound'],
  'Quantity': 1.0,
  'Ingredient Name': 'ground beef'},
 '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '],
  'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'bread crumbs'},
 '3/4 cup finely chopped onion': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'finely onion'},
 '3/4 cup chopped fresh basil': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'fresh basil'},
 '1/2 cup ketchup': {'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'ketchup'},
 '2 ounces shredded sharp provolone cheese': {'Preparation': 

In [5]:
healthy = ['wheat', 'cottage','sauce']
health_dict = [i for i in health_dict if not (i['unhealthy'] == 'cheese')] 
health_dict = [i for i in health_dict if not (i['unhealthy'] == 'parmesan cheese')] 
health_dict = [i for i in health_dict if not (i['unhealthy'] == 'mozzarella cheese')] 

cheese1 = {'healthy': 'soy parmesan', 'unhealthy': 'parmesan cheese'}
cheese2 = {'healthy': 'non-fat mozzarella', 'unhealthy': 'mozzarella cheese'}
health_dict.append(cheese1)
health_dict.append(cheese2)
#print(health_dict)
arr =[]
arr2 =[]

def to_Healthy(recipe_dic):
    transformed_directions =[]
    for i in recipe_dic:
        print(recipe_dic[i]['Ingredient Name'])
        curr = recipe_dic[i]['Ingredient Name'].lower()
        
        #print(dic[i]['Ingredient Name'])
        for j in health_dict:
            if (j['unhealthy']) in curr and  not(any(substring in curr for substring in healthy)):
                
                recipe_dic[i]['Ingredient Name'] = j['healthy']
                arr.append(curr)
                arr2.append(j['healthy'])
    
#     print(arr)
#     print(arr2)
    
    
    for k in recipe['directions']:
        
        fl = False
        for i in range(len(arr)):
            j = k.lower()
            if arr[i] in j:
                k =j.replace(arr[i],arr2[i])
#                 print(k)
        transformed_directions.append(k)
    return transformed_directions

In [6]:
update_dir = to_Healthy(dic)
print(update_dir)
print(recipe['directions'])

sun-tomatoes
hot water
cooking spray
ground beef
bread crumbs
finely onion
fresh basil
ketchup
sharp provolone cheese
egg whites
garlic
['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.', 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.', 'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together in a bowl; pack into prepared loaf pan.', 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']
['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.', 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.', 'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together 

In [7]:
def to_Unhealthy(recipe_dic):
    transformed_directions =[]
    for i in recipe_dic:
        curr = recipe_dic[i]['Ingredient Name'].lower()
        #print(dic[i]['Ingredient Name'])
        for j in health_dict:
            if (j['healthy']) in curr:
                recipe_dic[i]['Ingredient Name'] = j['unhealthy']
                arr.append(curr)
                arr2.append(j['unhealthy'])
#     print(arr)
#     print(arr2)
    
    
    for k in recipe['directions']:
        for i in range(len(arr)):
            j = k.lower()
            if arr[i] in j:
                k =j.replace(arr[i],arr2[i])
                #print(k)
        transformed_directions.append(k)
    return transformed_directions

In [8]:
recipe['directions']=  to_Unhealthy(dic)
print(recipe['directions'])
print(update_dir)
dic

['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.', 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.', 'mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, eggs, and garlic together in a bowl; pack into prepared loaf pan.', 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']
['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.', 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.', 'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together in a bowl; pack into prepared loaf pan.', 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermo

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '],
  'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'sun-tomatoes'},
 '1 cup hot water': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'hot water'},
 'cooking spray': {'Ingredient Name': 'cooking spray'},
 '1 pound ground beef': {'Measurement': ['pound'],
  'Quantity': 1.0,
  'Ingredient Name': 'ground beef'},
 '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '],
  'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'bread crumbs'},
 '3/4 cup finely chopped onion': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'finely onion'},
 '3/4 cup chopped fresh basil': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'fresh basil'},
 '1/2 cup ketchup': {'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'ketchup'},
 '2 ounces shredded sharp provolone cheese': {'Preparation': 

In [9]:
arr =[]
arr2 =[]
steps=[]

def to_Indian(recipe_dic):
    transformed_directions =[]
    for i in recipe_dic:
        curr = recipe_dic[i]['Ingredient Name'].lower()
        #print(dic[i]['Ingredient Name'])
        for a in style_dict:
            if a in curr:
                k = curr.replace(a, 'indian')
                curr = k
        for j in indian_dict:
            if (j['other']) in curr:
                curr = curr.replace(j['other'], j['indian'])
        recipe_dic[i]['Ingredient Name'] = curr
        arr.append(curr)
    while '' in arr: arr.remove('')
    print("INGREDIENTS:")
    print(arr)
    print("______________________________________________________")
    for k in recipe['directions']:
        fl = False
        j = k.lower()   
        for a in style_dict:
            if a in j:
                k = j.replace(a, 'indian')
                j = k
        for a in indian_dict:
            if a['other'] in j:
                k = j.replace(a['other'], a['indian'])
                j = k
                #print(a['other'])
                #print(k)
        transformed_directions.append(k)
        
    for k in transformed_directions:
        fl = False
        j = k.lower()   
        for a in indian_dict:   
            for word in a['other'].split():
                if word in j:
                    k = j.replace(word, a['indian'])
                    j = k
                    #print(word)
                    #print(k)
        steps.append(k)
    #print("DIRECTIONS:")
    #print(steps)
    return steps


In [10]:
def cut_or_double(direction,n):   
    updated_direc =[]
    for i in direction:
            x=re.findall('(\d+) \w*\s*minutes',i)
            if x!=[]:
                for j in x:
                    k = float(j)
                    k = k*n
                    k = str(k)
                    k = re.sub('\.\d+','',k)
                    i=re.sub(j,k,i)
            updated_direc.append(i)           
    return updated_direc

In [11]:
def to_double(recipe_dic):
    for i in recipe_dic:
        try:
            recipe_dic[i]['Quantity'] = 2*recipe_dic[i]['Quantity']
        except:
            pass
    update_dir1 = cut_or_double(update_dir,1.5)
    return update_dir1

In [12]:
update_dir = to_double(dic)
print(dic)
update_dir

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '], 'Measurement': ['cup'], 'Quantity': 1.0, 'Ingredient Name': 'sun-tomatoes'}, '1 cup hot water': {'Measurement': ['cup'], 'Quantity': 2.0, 'Ingredient Name': 'hot water'}, 'cooking spray': {'Ingredient Name': 'cooking spray'}, '1 pound ground beef': {'Measurement': ['pound'], 'Quantity': 2.0, 'Ingredient Name': 'ground beef'}, '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '], 'Measurement': ['cup'], 'Quantity': 2.0, 'Ingredient Name': 'bread crumbs'}, '3/4 cup finely chopped onion': {'Preparation': ['chopped '], 'Measurement': ['cup'], 'Quantity': 1.5, 'Ingredient Name': 'finely onion'}, '3/4 cup chopped fresh basil': {'Preparation': ['chopped '], 'Measurement': ['cup'], 'Quantity': 1.5, 'Ingredient Name': 'fresh basil'}, '1/2 cup ketchup': {'Measurement': ['cup'], 'Quantity': 1.0, 'Ingredient Name': 'ketchup'}, '2 ounces shredded sharp provolone cheese': {'Preparation': ['shredded '], 'Measurement': ['ounces'], 'Qua

['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 45 minutes; drain.',
 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
 'mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, eggs, and garlic together in a bowl; pack into prepared loaf pan.',
 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']

In [13]:
def to_half(recipe_dic):
    for i in recipe_dic:
        try:
            recipe_dic[i]['Quantity'] = 0.5*recipe_dic[i]['Quantity']
        except:
            pass
    update_dir1 = cut_or_double(update_dir,0.75)
    return update_dir1

In [14]:
update_dir = to_half(dic)

In [15]:
print(dic)
update_dir

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '], 'Measurement': ['cup'], 'Quantity': 0.5, 'Ingredient Name': 'sun-tomatoes'}, '1 cup hot water': {'Measurement': ['cup'], 'Quantity': 1.0, 'Ingredient Name': 'hot water'}, 'cooking spray': {'Ingredient Name': 'cooking spray'}, '1 pound ground beef': {'Measurement': ['pound'], 'Quantity': 1.0, 'Ingredient Name': 'ground beef'}, '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '], 'Measurement': ['cup'], 'Quantity': 1.0, 'Ingredient Name': 'bread crumbs'}, '3/4 cup finely chopped onion': {'Preparation': ['chopped '], 'Measurement': ['cup'], 'Quantity': 0.75, 'Ingredient Name': 'finely onion'}, '3/4 cup chopped fresh basil': {'Preparation': ['chopped '], 'Measurement': ['cup'], 'Quantity': 0.75, 'Ingredient Name': 'fresh basil'}, '1/2 cup ketchup': {'Measurement': ['cup'], 'Quantity': 0.5, 'Ingredient Name': 'ketchup'}, '2 ounces shredded sharp provolone cheese': {'Preparation': ['shredded '], 'Measurement': ['ounces'], 'Q

['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 33 minutes; drain.',
 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
 'mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, eggs, and garlic together in a bowl; pack into prepared loaf pan.',
 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']

In [16]:
update_dir = to_Healthy(dic)

sun-tomatoes
hot water
cooking spray
ground beef
bread crumbs
finely onion
fresh basil
ketchup
sharp provolone cheese
eggs
garlic


In [17]:
dic

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '],
  'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'sun-tomatoes'},
 '1 cup hot water': {'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'hot water'},
 'cooking spray': {'Ingredient Name': 'cooking spray'},
 '1 pound ground beef': {'Measurement': ['pound'],
  'Quantity': 1.0,
  'Ingredient Name': 'ground beef'},
 '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '],
  'Measurement': ['cup'],
  'Quantity': 1.0,
  'Ingredient Name': 'bread crumbs'},
 '3/4 cup finely chopped onion': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'finely onion'},
 '3/4 cup chopped fresh basil': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.75,
  'Ingredient Name': 'fresh basil'},
 '1/2 cup ketchup': {'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'ketchup'},
 '2 ounces shredded sharp provolone cheese': {'Preparation': 

In [18]:
#update_dir = to_Healthy(dic)
update_dir = to_half(dic)

In [19]:
update_dir

['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 22 minutes; drain.',
 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
 'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together in a bowl; pack into prepared loaf pan.',
 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']

In [20]:
update_dir = to_Indian(dic)

INGREDIENTS:
['eggs', 'sun-tomatoes', 'hot water', 'cooking spray', 'lamb or goat', 'bread crumbs', 'finely onion', 'fresh cilantro', 'tomato puree', 'sharp curd', 'egg whites', 'garlic']
______________________________________________________


In [21]:
update_dir

['put tomatoes in a bowl. pour bun water over tomatoes and soak until softened, about 30 minutes; drain.',
 'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
 'mix drained tomatoes, lamb or goat, bread crumbs, onion, cilantro, tomato puree, curd, egg whites, and garlic together in a bowl; pack into prepared loaf pan.',
 'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).']

In [22]:
dic

{'1/2 cup sun-dried tomatoes': {'Preparation': ['dried '],
  'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'sun-tomatoes'},
 '1 cup hot water': {'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'hot water'},
 'cooking spray': {'Ingredient Name': 'cooking spray'},
 '1 pound ground beef': {'Measurement': ['pound'],
  'Quantity': 0.5,
  'Ingredient Name': 'lamb or goat'},
 '1 cup seasoned bread crumbs': {'Preparation': ['seasoned '],
  'Measurement': ['cup'],
  'Quantity': 0.5,
  'Ingredient Name': 'bread crumbs'},
 '3/4 cup finely chopped onion': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.375,
  'Ingredient Name': 'finely onion'},
 '3/4 cup chopped fresh basil': {'Preparation': ['chopped '],
  'Measurement': ['cup'],
  'Quantity': 0.375,
  'Ingredient Name': 'fresh cilantro'},
 '1/2 cup ketchup': {'Measurement': ['cup'],
  'Quantity': 0.25,
  'Ingredient Name': 'tomato puree'},
 '2 ounces shredded sharp provolone cheese': {'P

In [23]:
for i in health_dict:
    if 'cheese' in i['unhealthy']:
        print(i['unhealthy'])

low-fat cheese
parmesan cheese
mozzarella cheese


In [24]:
import numpy as np

primary_cooking = json_data["methods"]["primary"]

primary_methods = []
for k in recipe['directions']:
    fl = False
    j = k.lower()   
    for a in primary_cooking:
        if a in j:
            primary_methods.append(a)
                
print(np.unique(primary_methods))

['bake' 'mix']


In [26]:
secondary_cooking = json_data["methods"]["secondary"]

secondary_methods = []
for k in recipe['directions']:
    fl = False
    j = k.lower()   
    for a in secondary_cooking:
        if a in j:
            secondary_methods.append(a)
                
print(np.unique(secondary_methods))

['bake' 'drain' 'heat' 'mix' 'pour' 'preheat' 'soak']


In [27]:

tools_db = json_data["tools"]

tools = []
for k in recipe['directions']:
    j = k.lower()   
    for a in tools_db:
        if a in j:
            tools.append(a)
                
print(np.unique(tools))

['bowl' 'oven' 'pan']


In [28]:
recipe['tools'] = np.unique(tools)
recipe['primary methods'] = np.unique(primary_methods)

In [29]:
recipe['secondary methods'] = np.unique(secondary_methods)

In [30]:
recipe

{'ingredients': ['1/2 cup sun-dried tomatoes',
  '1 cup hot water',
  'cooking spray',
  '1 pound ground beef',
  '1 cup seasoned bread crumbs',
  '3/4 cup finely chopped onion',
  '3/4 cup chopped fresh basil',
  '1/2 cup ketchup',
  '2 ounces shredded sharp provolone cheese',
  '2 egg whites',
  '2 cloves garlic, chopped'],
 'labels': [],
 'directions': ['Put tomatoes in a bowl. Pour hot water over tomatoes and soak until softened, about 30 minutes; drain.',
  'Preheat oven to 350 degrees F (175 degrees C). Prepare a loaf pan with cooking spray.',
  'Mix drained tomatoes, ground beef, bread crumbs, onion, basil, ketchup, provolone cheese, egg whites, and garlic together in a bowl; pack into prepared loaf pan.',
  'Bake in the preheated oven until no longer pink in the center, about 1 hour. An instant-read thermometer inserted into the center should read at least 160 degrees F (70 degrees C).'],
 'tools': array(['bowl', 'oven', 'pan'], dtype='<U4'),
 'primary methods': array(['bake', 

'bowl'